In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3 as db

In [2]:
#Get sample data
df = pd.read_csv(r'C:\Projects\Strava\RideData_20150628145227.csv')
df.head(1)

,act_startDate,timestamp,time,act_id,act_name,distance,altitude,velocity_smooth,moving,grade_smooth,timestamp2,lat,long
0,2015-06-28 11:36:16,2015-06-28 11:36:16+00:00,0,334723309,Morning Ride,0.0,180.2,0.0,False,0.0,2015-06-28 11:36:16,44.615305,-88.039688


In [3]:
#Create our test database for upserts
conn = db.connect('test_upsert.db')
engine = create_engine(r'sqlite:///test_upsert.db')

In [4]:
#Create a table using pandas to_sql
df.to_sql('test_upsert', engine, if_exists='append')

In [5]:
#Check that the table exists and there is data in it
df_in_db = pd.read_sql_query('SELECT * FROM test_upsert', engine)
df_in_db.head(1)

,index,act_startDate,timestamp,time,act_id,act_name,distance,altitude,velocity_smooth,moving,grade_smooth,timestamp2,lat,long
0,0,2015-06-28 11:36:16,2015-06-28 11:36:16+00:00,0,334723309,Morning Ride,0.0,180.2,0.0,0,0.0,2015-06-28 11:36:16,44.615305,-88.039688


In [6]:
#Now lets bring in some new data to insert, along with the same old data
df2 = pd.concat([pd.read_csv(r'C:\Projects\Strava\RideData_20150628145227.csv'),
               pd.read_csv(r'C:\Projects\Strava\RideData_20150628145421.csv')])
df2.head(1)

,act_id,act_name,act_startDate,altitude,distance,grade_smooth,lat,long,moving,time,timestamp,timestamp2,velocity_smooth
0,334723309,Morning Ride,2015-06-28 11:36:16,180.2,0.0,0.0,44.615305,-88.039688,False,0,2015-06-28 11:36:16+00:00,2015-06-28 11:36:16,0.0


In [7]:
#First let's get the length of both dataframe
len_df2 = df2.shape[0]
len_df_in_db = df_in_db.shape[0]
print ('new df is %s rows, and data in db is %s rows') %(len_df2, len_df_in_db)

new df is 11806 rows, and data in db is 11806 rows


In [26]:
#Now let's find out what rows are duplicates using a combination of full outer join and select where is not null
df_common = pd.merge(df2, df_in_db, how='inner', left_on=['act_id', 'timestamp'], right_on=['act_id', 'timestamp'])
print 'outer joined df is %s rows' %(df_common.shape[0])

MemoryError: 

In [ ]:
import gc
gc.collect()

In [25]:
df_common['distance'].isnull().describe()

NameError: name 'df_common' is not defined

In [18]:
#now let's get rid of duplicate rows!
df_all = df_all.dropna()
print df_all.shape[0]

17709
